In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cardataset/data.csv


In [3]:
!pip install pyspark

     |████████████████████████████████| 212.3 MB 14 kB/s s eta 0:00:01
     |████████████████████████████████| 198 kB 35.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=d0fc92c100a0aa652d396699e197305817d4b251a262c8b9a45ebec6d65ecc9d
  Stored in directory: /root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark


In [4]:
!pip show pyspark

Name: pyspark
Version: 3.1.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: py4j
Required-by: 


In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

In [6]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
data = sc.read.csv('/kaggle/input/cardataset/data.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



In [9]:
data.toPandas()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [10]:
data.toPandas().isnull().sum().sort_values(ascending=False)

Engine HP            69
Engine Cylinders     30
Number of Doors       6
Engine Fuel Type      3
Make                  0
Model                 0
Year                  0
Transmission Type     0
Driven_Wheels         0
Market Category       0
Vehicle Size          0
Vehicle Style         0
highway MPG           0
city mpg              0
Popularity            0
MSRP                  0
dtype: int64

In [11]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|              0|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [12]:
data = data.drop('Market Category')
data = data.na.drop()
print((data.count(), len(data.columns)))

(11812, 15)


In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor

In [27]:
assembler = VectorAssembler(inputCols=['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors', 'highway MPG', 'city mpg', 'Popularity'],
                           outputCol='Attributes')

regressor = RandomForestRegressor(featuresCol='Attributes', labelCol='MSRP')

In [28]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, regressor])
pipeline.write().overwrite().save("pipeline")
!ls

__notebook_source__.ipynb  pipeline


In [26]:
pipelineModel = Pipeline.load("pipeline")

In [29]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)
model = pipelineModel.fit(train_data)
predictions = model.transform(test_data)

In [32]:
# Select example rows to display.
predictions.select("MSRP", "prediction").show()

+-----+------------------+
| MSRP|        prediction|
+-----+------------------+
|28030|  33839.3752908075|
|30550| 36852.29041033867|
|29350| 27490.31552338961|
|27900|   26895.026207626|
|34890|   26895.026207626|
|32990|   26895.026207626|
| 2827|5875.3673242829755|
| 3000|5875.3673242829755|
| 3086| 5820.469833165014|
| 3130| 5820.469833165014|
| 3012| 5888.395181397732|
| 3622| 5896.958682185754|
|22300| 23168.34483735251|
|19400|22199.644688793724|
| 2042| 5082.834946088396|
| 2144| 3942.085812686078|
|49440|41284.920379312956|
|52640|41284.920379312956|
|47440| 41724.36507203108|
|58400| 41724.36507203108|
+-----+------------------+
only showing top 20 rows



In [34]:
from pyspark.ml.evaluation import RegressionEvaluator

In [37]:
evaluator = RegressionEvaluator(labelCol="MSRP", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print("RMSE ", rmse)

RMSE  32177.513051853275
